In [1]:
import matplotlib.pyplot as plt
import numpy as np
import re
import json
import sys 
import os
import random


import scipy.special
from scipy.stats import linregress
from scipy.spatial.distance import jaccard
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

from sklearn.decomposition import KernelPCA
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.manifold import MDS

from hdbscan import HDBSCAN

from scipy.cluster import hierarchy as hrc
from sklearn.decomposition import PCA

from sklearn.cluster import AgglomerativeClustering

from sklearn.metrics import jaccard_similarity_score
jaccard_index = jaccard_similarity_score

import copy

sys.path.append("./src/")
import lumberjack
import tree_reader as tr



/Users/boris/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
johnston_counts = np.loadtxt("/Users/boris/taylor/johnston_retina/single_cell/dmel-retina-scRNA/exploration/2018.07.19_Scanpy/counts.txt")
johnston_header = np.loadtxt("/Users/boris/taylor/johnston_retina/single_cell/dmel-retina-scRNA/exploration/2018.07.19_Scanpy/header.txt",dtype=str)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/boris/taylor/vision/rust_prototype/2_forest/trees/johnston_retina/deeper/run.0.compact'

In [ ]:
print(counts.shape)
print(header.shape)

In [ ]:
counts[:10,:10]
np.var(counts,axis=0).shape

In [ ]:
# sample_encoding = node_sample_encoding(first_forest.leaves(),first_forest.dim[0]).T
# print(sample_encoding.shape)

# distances = coocurrence_distance(sample_encoding)
# print(distances.shape)
# embedding_model = MDS(n_components=2,dissimilarity='precomputed',metric=False)
# coordinates = embedding_model.fit_transform(distances)

# distances = coocurrence_matrix(sample_encoding)
# print(distances.shape)
# embedding_model = KernelPCA(n_components=2,kernel='precomputed')
# coordinates = embedding_model.fit_transform(distances)



# print(coordinates.shape)

# plt.figure()
# plt.scatter(coordinates[:,0],coordinates[:,1],s=.1)
# plt.show()

In [ ]:
tsne_coordinates = sample_tsne(first_forest.leaves(),first_forest.dim[0])

# hd_clusters = sample_hdbscan(first_forest.leaves(),first_forest.dim[0])
# a_clusters = sample_agglomerative(first_forest.leaves(),first_forest.dim[0],13)
l_clusters = hacked_louvain(first_forest.leaves(),first_forest.samples)
# e_clusters = embedded_hdbscan(tsne_coordinates)

plt.figure()
plt.scatter(tsne_coordinates[:,0],tsne_coordinates[:,1],s=.5,c=(l_clusters+1),cmap='rainbow')
# plt.scatter(tsne_coordinates[:,0],tsne_coordinates[:,1],s=.5)
plt.show()


In [30]:
# sample_encoding = node_sample_encoding(first_forest.leaves(),first_forest.dim[0]).T

print(sample_encoding.shape)

# np.savetxt("/Users/boris/haxx/python/sample_encoding.txt",sample_encoding)

# print(squareform(pdist(sample_encoding,metric='cityblock')).shape)

# plt.figure()
# plt.hist(pdist(sample_encoding,metric='cityblock'),bins=np.arange(0,100,2))
# plt.show()

(2585, 3228)


In [ ]:

def cascading_node_slices_x(node,depth=0):

    splits = []
    
    leaves = node.leaves()
    
    print "Cascading"
    print len(leaves)
    
    total_counts = np.zeros((0,len(node['features'])))

    for leaf in leaves:
        if len(leaf['children']) > 0:
            node_counts = leaf.singly_sorted_counts()
            splits.append(total_samples + split_index)
        else:
            node_counts = 
        total_samples += node_counts.shape[0]
        total_counts = np.concatenate((total_counts,node_counts), axis=0)
    
    return total_counts, splits


    
def make_tree_movie(node,counts,header,feature_sort,title,max_depth=1):

    splits = []
    
    for i in range(max_depth):
        
        split_map, new_splits = cascading_node_slices_x(node,counts,header,depth=i)

        plt.figure(figsize=(8,4))
        im = plt.imshow(split_map.T[feature_sort].T,aspect='auto')
        for split in splits:
            plt.plot([0,split_map.shape[1]-1],[split,split],c='xkcd:cyan',linewidth=.25)
        for split in new_splits:
            plt.plot([0,split_map.shape[1]-1],[split,split],c='blue',linewidth=.25)
        splits.extend(new_splits)
        plt.savefig('figures/tree_movie/%s.png' % (title + "." + str(i)),dpi=500)

           
node_counts = node_to_naive_counts(first_forest[0],counts,header)

gene_linked = hrc.linkage(node_counts.T, method='average', metric='correlation')
gene_dendrogram = hrc.dendrogram(gene_linked,no_plot=True)

# make_tree_movie(first_forest[0],counts,header,gene_dendrogram['leaves'],"tree1",max_depth=5)

In [ ]:
print len([x for y in tree_to_leaves(first_forest[0],0,) for x in y['samples']])

# combined_2nd_tree_files = open('trees_for_clustering/combined/2nd_paths.txt').readlines()

# second_layer = []

# for tree_file in combined_2nd_tree_files:
#     second_layer.append(json.load(open(tree_file.strip())))


In [ ]:
first_forest_leaves = forest_to_leaves(first_forest)
# second_forest_leaves = forest_to_leaves(second_layer)

print len(first_forest_leaves)
# print len(second_forest_leaves)

# print [len(x['features']) for x in first_forest_leaves]

# print list(alt_node_sample_encoding(first_forest_leaves)[0])
# print list(first_forest_leaves[0]['samples'])

# print np.sum(alt_node_sample_encoding(first_forest_leaves)[0])
# print len(first_forest_leaves[0]['samples'])

# for sample in first_forest_leaves[0]['samples']:
#     print alt_node_sample_encoding(first_forest_leaves)[0][int(sample)]


# print len(first_forest_leaves[0]['samples'])

ff_sample_coordinates = node_sample_tsne(first_forest_leaves)
# sf_sample_coordinates = node_sample_mds(second_forest_leaves)

print "Tsne is done"

ff_sample_feature_cross_clusters = node_sample_hdbscan(first_forest_leaves)
# fs_sample_feature_cross_clusters = node_feature_dbscan(second_forest_leaves)

# print ff_sample_coordinates.shape
# print sf_sample_coordinates.shape

plt.figure("first_forest_sample_pca")
plt.title("First Layer Forest Node Sample Membership")
plt.scatter(ff_sample_coordinates[:,0],ff_sample_coordinates[:,1],s=.03,c=ff_sample_feature_cross_clusters,cmap="rainbow")
plt.savefig(prefix + "ff_sample_scatter.png", dpi=300)

# plt.figure("second_forest_sample_pca")
# plt.title("Second Layer Forest Node Sample Membership")
# plt.scatter(sf_sample_coordinates[:,0],sf_sample_coordinates[:,1],s=.03,c=fs_sample_feature_cross_clusters)
# plt.show()



In [ ]:
# impute_error = np.loadtxt('trees_for_clustering/combined/impute_error.txt')

In [ ]:
# plt.figure()
# plt.hist([np.sum(clusters == cluster) for cluster in set(list(clusters))],bins=50,log=True)
# plt.show()

In [ ]:
# plt.figure("embedded_nodes")
# plt.scatter(coordinates[:,0],coordinates[:,1],s=.03,c=clusters,cmap='rainbow')
# plt.savefig("node_clusters.png",dpi=500)

In [ ]:
# clst = coordinates[:,0] > 18

In [ ]:
# print sum(clst)

In [ ]:
# clst_indecies = np.arange(len(forest_nodes))[clst]
# clst_nodes = []

# for ind in clst_indecies:
#     clst_nodes.append(forest_nodes[ind])

# requirements = [x['requirements'] for x in clst_nodes]



In [ ]:
# [len(x['samples']) for x in clst_nodes]

In [ ]:
# plt.figure()
# plt.hist([x[0] for y in x for x in requirements])
# plt.show()

In [ ]:
# import seaborn as sns

In [ ]:
# large_cluster_requirements = [forest_nodes[x]['requirements'] for x in clusters == 0]

# print large_cluster_requirements[:300]
# plt.figure()
# plt.hist([x[0] for x in large_cluster_requirements if len(x) > 0])
# plt.show()

In [ ]:
# print [x['requirements'] for x in first_forest][:5]

In [ ]:
def extract_absolute_gain_vectors(nodes):
    
    features = {}
    
    gain_pairs = []
    
    for node in nodes:
        node_gain_pairs = []
        try:
            for feature,gain in zip(node['features'],node['absolute_gains']):
                if feature not in features:
                    features[feature] = len(features)
                node_gain_pairs.append((features[feature],gain))
            gain_pairs.append(node_gain_pairs)
        except:
            continue
    
    gain_matrix = np.zeros((len(nodes),len(features)))
    
    for n_ind,node_gain_pairs in enumerate(gain_pairs):
        for f_ind,gain in node_gain_pairs:
            gain_matrix[n_ind,f_ind] = gain
    
    features = [(x,features[x]) for x in features.keys()]
    
    features = sorted(features,key=lambda x: x[1])
    
    return gain_matrix, features

def extract_local_gain_vectors(nodes):
    
    features = {}
    
    gain_pairs = []
    
    for node in nodes:
        node_gain_pairs = []
        try:
            for feature,gain in zip(node['features'],node['local_gains']):
                if feature not in features:
                    features[feature] = len(features)
                node_gain_pairs.append((features[feature],gain))
            gain_pairs.append(node_gain_pairs)
        except:
            continue
    
    gain_matrix = np.zeros((len(nodes),len(features)))
    
    for n_ind,node_gain_pairs in enumerate(gain_pairs):
        for f_ind,gain in node_gain_pairs:
            gain_matrix[n_ind,f_ind] = gain
    
    return gain_matrix


def node_feature_pca(nodes):

    node_encoding,feature_header = extract_absolute_gain_vectors(nodes)

    embedding_model = PCA(n_components=2)

    coordinates = embedding_model.fit_transform(node_encoding)

    return coordinates



def node_feature_kpca(nodes):

    node_encoding,feature_header = extract_absolute_gain_vectors(nodes)

    embedding_model = KernelPCA(n_components=2,kernel='correlation')

    coordinates = embedding_model.fit_transform(node_encoding)

    return coordinates

def node_feature_tsne(nodes):

    node_encoding,feature_header = extract_absolute_gain_vectors(nodes)

#     embedding_model = TSNE(n_components=2,early_exaggeration=3,learning_rate=5,metric='correlation')

    embedding_model = TSNE(n_components=2, metric='cosine')


    coordinates = embedding_model.fit_transform(node_encoding)

    return coordinates

def node_feature_mds(nodes):
    
    node_encoding,feature_header = extract_absolute_gain_vectors(nodes)
    
    embedding_model = MDS(n_components=2,dissimilarity='precomputed')
        
    distances = scipy.spatial.distance.squareform(pdist(node_encoding,metric='correlation'))
    
    coordinates = embedding_model.fit_transform(distances)
    
    return coordinates

def node_feature_dbscan(nodes):

    node_encoding,feature_header = extract_absolute_gain_vectors(nodes)

    distances = scipy.spatial.distance.squareform(pdist(node_encoding,metric='cosine'))

    clustering_model = DBSCAN(min_samples=10, eps = .35, metric='precomputed')
    
    clusters = clustering_model.fit_predict(distances)
    
    plt.figure()
    plt.title("Dbscan observed distances")
    plt.hist(distances.flatten(),bins=50)
    plt.show()
    
    return clusters


In [ ]:
first_forest_leaves = forest_to_leaves(first_forest)
# second_forest_leaves = forest_to_leaves(second_layer)

ff_feature_coordinates = node_feature_tsne(first_forest_leaves)

ff_feature_clusters = node_feature_dbscan(first_forest_leaves)

# fs_feature_coordinates = node_feature_mds(second_forest_leaves)

# fs_feature_clusters = node_feature_dbscan(second_forest_leaves)

plt.figure("first_forest_feature_pca")
plt.title("First Layer Forest Leaf Feature Gain")
plt.scatter(ff_feature_coordinates[:,0],ff_feature_coordinates[:,1],s=.03,c=ff_feature_clusters,cmap='rainbow')
plt.savefig(prefix + "ff_feature_tsne_dbscan_color.png",dpi=500)

# plt.figure("second_forest_feature_pca")
# plt.title("Second Layer Forest Node Feature Gain")
# plt.scatter(fs_feature_coordinates[:,0],fs_feature_coordinates[:,1],s=.03,c=fs_feature_clusters,cmap='rainbow')
# plt.savefig("figures/fs_feature_tsne_dbscan_color.png",dpi=300)


In [ ]:
# print len(set(list(fs_feature_clusters)))

In [ ]:
# dist = pdist(extract_local_gain_vectors(forest_to_nodes(first_forest)),metric='cityblock')
# print dist.shape
# print dist[:10]
# print np.sum(np.isnan(dist))
# plt.figure("cos_dist_dist")
# plt.hist(dist,bins=50)
# plt.show()

In [ ]:
# first_forest_nodes = forest_to_nodes(first_forest)
# # second_forest_nodes = forest_to_nodes(second_layer)

# ff_feature_vectors = extract_local_gain_vectors(first_forest_nodes)

# embedding_model = TSNE(n_components=2,metric='cosine',learning_rate=100,perplexity=50)
# ff_local_feature_coordinates = embedding_model.fit_transform(ff_feature_vectors)

# clustering_model = DBSCAN(eps=50,min_samples=2,metric='cityblock')
# ff_local_feature_clusters = clustering_model.fit_predict(ff_feature_vectors)

# plt.figure("first_forest_feature_pca")
# plt.title("First Layer Forest Node Feature Gain")
# plt.scatter(ff_local_feature_coordinates[:,0],ff_local_feature_coordinates[:,1],s=.03,c=ff_local_feature_clusters,cmap='rainbow')
# plt.savefig("figures/ff_feature_tsne_dbscan_color.png",dpi=500)

# plt.figure("second_forest_feature_pca")
# plt.title("Second Layer Forest Node Feature Gain")
# plt.scatter(fs_feature_coordinates[:,0],fs_feature_coordinates[:,1],s=.03,c=fs_feature_clusters,cmap='rainbow')
# plt.savefig("figures/fs_feature_tsne_dbscan_color.png",dpi=300)

In [ ]:
def cluster_stats(nodes):
    
    split_features = [x['feature'] for x in nodes]

    samples_per_node = [len(x['samples']) for x in nodes]

    cluster_jaccard = scipy.spatial.distance.squareform(pdist(node_sample_encoding(nodes),metric='jaccard'))
    
    requirements = [x for y in nodes for x in zip(y['requirements'][::2],y['requirements'][1::2])]
    
    requirements_set = set([x[0] for x in requirements])
    
    requirement_frequencies = []
    
    for requirement in requirements_set:
        
        requirement_frequencies.append((requirement, sum([int(x[0] == requirement) for x in requirements])))
    
    requirement_frequencies = sorted(requirement_frequencies,key=lambda x: x[1])
    
    feature_gains, gains_header = extract_absolute_gain_vectors(nodes)

    tree_ids = [x['tree_id'] for x in nodes]

    mean_gains = np.mean(feature_gains, axis = 0)
    
    feature_gains_sort_indecies = np.argsort(mean_gains)
    
    
    sorted_feature_gains = feature_gains.T[feature_gains_sort_indecies].T
    sorted_features = np.array(gains_header)[feature_gains_sort_indecies]
    
    print "Nodes: " + str(len(nodes))
    
    print "Top features"
    print sorted_features[-10:]
    print np.mean(sorted_feature_gains,axis=0)[-10:]
    
    print "Top Requirements:"
    print requirement_frequencies[-10:]
    
    plt.figure()
    plt.hist(tree_ids)
    plt.show()
    
    
    
for cluster in set(list(ff_sample_feature_cross_clusters)):
    
    indecies = np.arange(first_forest_leaves.shape[0])[ff_feature_clusters == cluster]
    
    cluster_nodes = []
    
    for index in indecies:
        
        cluster_nodes.append(first_forest_leaves[index])
    
    cluster_stats(cluster_nodes)
    
    